In [123]:
import numpy as np
from collections import defaultdict
import heapq
from pqdict import pqdict

In [124]:
def tie_breaking_precedes(a, b):
    # a, b are (priority, key) tuples
    # Prefer smaller priority first; if equal, prefer smaller key
    return a[0] < b[0] or (a[0] == b[0] and a[1] < b[1])

In [125]:
vertices = [1,2,3,4,5,6,7]
edges = defaultdict(dict)
edges[1][3] = 5
edges[1][5] = 2
edges[1][6] = 5
edges[2][5] = 9
edges[2][6] = 1
edges[3][4] = 1
edges[3][5] = 1
edges[5][6] = 4
edges[5][3] = 1
edges[6][7] = 5
edges[6][1] = 5
edges[7][4] = 5


# heuristic
heuristic = {}
heuristic[1] = 1
heuristic[2] = 10
heuristic[3] = 3
heuristic[4] = 0
heuristic[5] = 2
heuristic[6] = 7
heuristic[7] = 5

start = 2
goal = 4

In [126]:
class Node(object):
    def __init__(self, key):
        self.key = key
        self.parent = None
        self.f = float('inf')
        self.g = float('inf')
        self.h = 0.0
        self.is_open = False
        self.is_closed = False
        self.children = []
        self.v = float('inf')

    def setParent(self, parent):
        self.parent = parent

    def setChildren(self, children):
        self.children = children

    def setG(self, g):
        self.g = g
        self.f = self.g + self.h

    def setHeuristic(self, h):
        self.h = h
        self.f = self.g + self.h

    def setV(self, v):
        self.v = v

    def getHeuristic(self):
        return self.h

    def isOpen(self):
        return self.is_open

    def isClosed(self):
        return self.is_closed

    def getParent(self):
        return self.parent

    def getG(self):
        return self.g

    def getF(self):
        self.f = self.g + self.h
        return self.f

In [127]:
class A_Star():
    def __init__(self, start, goal, heuristic, edges, vertices, step=4, epsilon=2):
        self.start = start
        self.goal = goal
        self.heuristic = heuristic
        self.step = step
        self.edges = edges
        self.vertices = vertices
        self.eps = epsilon

    def initialize(self, start):
        open_heap = pqdict(precedes=tie_breaking_precedes).minpq()
        nodes = {}
        for v in self.vertices:
            node = Node(v)
            node.setHeuristic(self.heuristic[v] * self.eps)
            node.setChildren(list(self.edges[v].keys()))
            if v == start:
                node.setG(0)
                node.is_open = True
                f = node.getF()
                open_heap[v] = (f, v)
            nodes[v] = node
        return open_heap, nodes

    def find_node(self, nodes, key):
        return nodes.get(key)

    def run(self):
        self.open_heap, self.node_dict = self.initialize(self.start)
        self.closed_list = []
        expanded_count = 0
        self.inconsist = []

        while self.open_heap and expanded_count < self.step:
            print("================================================")
            print(f"Iteration {expanded_count}")
            # Get the node with the lowest f-score
            item = self.open_heap.popitem()
            current_key = item[0]
            current_f = item[1][0]
            current_node = self.node_dict[current_key]
            current_node.is_open = False
            current_node.is_closed = True
            self.closed_list.append(current_key)
            current_node.setV(current_node.getG())
            print(f"Node exiting OPEN: {current_key}")

            # Check if goal reached
            if current_key == self.goal:
                return self.open_heap, self.closed_list, self.node_dict, self.inconsist, True

            # expand node
            for child_key, edge_cost in self.edges[current_key].items():
                child_node = self.node_dict[child_key]

                tentative_g = current_node.getG() + edge_cost
                if tentative_g < child_node.getG():
                    child_node.setParent(current_node)
                    child_node.setG(tentative_g)

                    # update
                    if child_key in self.closed_list:
                        self.inconsist.append(child_key)
                    else:
                        f = child_node.getF()
                        self.open_heap[child_key] = (f, child_key)
                        child_node.is_open = True
            print(f"OPEN: {list(self.open_heap.keys())}")
            for key, node in self.node_dict.items():
                print(f"Node {key}: {node.getG()}")

            expanded_count += 1
        return self.open_heap, self.closed_list, self.node_dict, self.inconsist, False

In [128]:
a_start = A_Star(start, goal, heuristic.copy(), edges, vertices, step=5, epsilon=2)
open_heap, close_list, node_dict, inconsist, is_goal = a_start.run()

Iteration 0
Node exiting OPEN: 2
OPEN: [5, 6]
Node 1: inf
Node 2: 0
Node 3: inf
Node 4: inf
Node 5: 9
Node 6: 1
Node 7: inf
Iteration 1
Node exiting OPEN: 5
OPEN: [6, 3]
Node 1: inf
Node 2: 0
Node 3: 10
Node 4: inf
Node 5: 9
Node 6: 1
Node 7: inf
Iteration 2
Node exiting OPEN: 6
OPEN: [1, 7, 3]
Node 1: 6
Node 2: 0
Node 3: 10
Node 4: inf
Node 5: 9
Node 6: 1
Node 7: 6
Iteration 3
Node exiting OPEN: 1
OPEN: [3, 7]
Node 1: 6
Node 2: 0
Node 3: 10
Node 4: inf
Node 5: 8
Node 6: 1
Node 7: 6
Iteration 4
Node exiting OPEN: 3
OPEN: [4, 7]
Node 1: 6
Node 2: 0
Node 3: 10
Node 4: 11
Node 5: 8
Node 6: 1
Node 7: 6


In [129]:
print(inconsist)

[5]
